In [ ]:
!pip install -q streamlit langchain langchain_experimental langchain-openai langchain-community langchain-text-splitters chromadb python-dotenv pyngrok pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 55.2 MB/s eta 0

In [ ]:
!ngrok config add-authtoken "31pHSQ5Ej8bCmgm2JaVih9RW9lg_6Byj5qALS13ejC2k5Wzwm"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os

# 직접 환경변수 등록 (가장 확실한 방법) 아래에 OPEN AI 키 넣기
os.environ["OPENAI_API_KEY"] = "N3WP2eZzcA"

# 확인
print("✅ OPENAI_API_KEY 등록 완료:", os.environ.get("OPENAI_API_KEY")[:10] + "...")

✅ OPENAI_API_KEY 등록 완료: sk-proj--7...


In [ ]:
# 1단계
%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RAG 챗봇 만들기 Step 1")
st.title("기본 LLM 챗봇 만들기")

# 세션 초기화
# 임시 메모리 -> 새로고침 전까지 유지되는 메모리 - 대화 내용이 없으면 빈 리스트!
if "messages" not in st.session_state:
    st.session_state["messages"] = []

# LLM 구성
# 1. AI의 목적 및 프롬프트
# 2. 사용자의 대화를 전달받는 q {q}
# 3. 답변을 비워놔서 AI답변을 기다린다.
llm = ChatOpenAI(model = 'gpt-4o-mini')
prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야.
사용자 : {q}
답변  :
""")
chain = prompt | llm | StrOutputParser()

# Streamlit 가장 중요한 요소 중 1개 chat_message - 스트림릿의 채팅 UI요소 뼈대
# session_state가 가지고 있는 메모리 -> 채팅UI 출력물을 뿌려줘야 하는데
# role 사람인지 챗봇인지 구별 -> content를 채팅 UI에 출력!
for m in st.session_state["messages"]:
    with st.chat_message(m['role']):
        st.markdown(m["content"])

# 알맹이 사용자 입력 -> LLM 응답 -> 렌더링(실시간 대화)
q = st.chat_input("질문을 입력해주세요") # input 설정
if q: #q가 질문이 입력이 되었다면
    with st.chat_message("user"):
        st.markdown(q) # q출력
    st.session_state['messages'].append({'role' : 'user', 'content' : q})

    with st.chat_message('assistant'):
        ans = chain.invoke({"q": q})
        st.markdown(ans)
    st.session_state['messages'].append({'role' : 'assistant', 'content': ans})

Overwriting app.py


In [ ]:
# 2단계 - RAG기반 챗봇 구현
%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # 2단계 추가
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader # 2단계 추가
from langchain_text_splitters import RecursiveCharacterTextSplitter # 2단계 추가
from langchain_community.vectorstores import Chroma # 2단계 추가

# 2단계에 필요한 라이브러리 추가

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RAG 챗봇 만들기 Step 2")
st.title("RAG 챗봇 만들기")

# 세션 초기화
# 임시 메모리 -> 새로고침 전까지 유지되는 메모리 - 대화 내용이 없으면 빈 리스트!
if "messages" not in st.session_state:
    st.session_state["messages"] = []

# LLM 구성
# 1. AI의 목적 및 프롬프트
# 2. 사용자의 대화를 전달받는 q {q}
# 3. 답변을 비워놔서 AI답변을 기다린다.
llm = ChatOpenAI(model = 'gpt-4o-mini')
prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야.
아래 문서 내용을 참고해 질문에 답하세요.
문서   : {context}
사용자 : {q}
답변  :
""")
chain = prompt | llm | StrOutputParser()

# RAG관련 문서 추가 - 수업에서는 하단에 있는 4줄 샘플로 진행
# 3) 문서→임베딩→DB (2교시 신규 추가)
os.makedirs("data", exist_ok=True)                                     # [추가]
doc_path = "data/정책.md"                                              # [추가]
if not os.path.exists(doc_path):                                       # [추가: 샘플 생성]
    with open(doc_path, "w", encoding="utf-8") as f:
        f.write(
            "- 디지털 상품은 원칙적으로 환불 불가 (결제 오류·중복 결제 예외)\n"
            "- 실물 상품은 수령 7일 이내 교환/환불 가능(미개봉)\n"
            "- VIP(GOLD) 고객은 1회 예외 환불 가능 (관리자 승인)\n"
            "- 배송 지연 3영업일 초과 시 사과 쿠폰 지급(내부 승인)\n"
        )

# 상단에 4줄 문장을 읽을 준비 완료
loader = TextLoader(doc_path, encoding = 'utf-8')
docs = loader.load()

# RAG관련 문서 정제
# 1) 문장을 청크(chunk)해서 나누고, 겹치게 구성
splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 100)
chunks = splitter.split_documents(docs)

# 2) 임베딩 - 문장을 숫자화
emb = OpenAIEmbeddings(model = 'text-embedding-3-small')
# 3) 임베딩된 문장을 Chroma DB에 저장! Vector DB에 저장
db  = Chroma.from_documents(chunks, emb)
# 향후 db에 있는 내용을 retriever로 가져오기만하면 되는 상황

# RAG기능 테스트! 간단 UI
# 4) streamlit UI 적용
st.write("RAG기반 문서 질의")
q = st.text_input("질문을 입력하세요", placeholder = "예: VIP환불정책을 알려주세요")

if q:
    # 1) 위에서 3가지 유사 문장을 가져온다.
    hits = db.similarity_search(q, k=3)
    # 2) 그 문장을 하나의 문장으로 병합한다.
    context = "\n\n".join(d.page_content for d in hits)
    # 3) 응답을 할 수 있게 구현한다.
    answer = chain.invoke({'context' : context, "q": q})

    st.markdown(f"답변 : {answer}")

Overwriting app.py


In [ ]:
# 3단계 - 키워드 기반 자동 라우팅 - 키워드에 따라 일반 대화 및 RAG 답변을 하는 챗봇 구현
%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # 2단계 추가
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader # 2단계 추가
from langchain_text_splitters import RecursiveCharacterTextSplitter # 2단계 추가
from langchain_community.vectorstores import Chroma # 2단계 추가

# 2단계에 필요한 라이브러리 추가

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RAG 챗봇 만들기 Step 3")
st.title("자동 라우팅 RAG 챗봇")

# 세션 초기화
# 임시 메모리 -> 새로고침 전까지 유지되는 메모리 - 대화 내용이 없으면 빈 리스트!
if "messages" not in st.session_state:
    st.session_state["messages"] = []

# LLM 구성
# 1. AI의 목적 및 프롬프트
# 2. 사용자의 대화를 전달받는 q {q}
# 3. 답변을 비워놔서 AI답변을 기다린다.
llm = ChatOpenAI(model = 'gpt-4o-mini')


# RAG 대화 전용 (이름 변경)
rag_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야.
아래 문서 내용을 참고해 질문에 답하세요.
문서   : {context}
사용자 : {q}
답변  :
""")
rag_chain = rag_prompt | llm | StrOutputParser()

# 일반 대화 전용 LCEL 구축 (일반 대화 - 추가)
base_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야
사용자 : {q}
답변  :
"""
)
base_chain = base_prompt | llm | StrOutputParser()


# RAG관련 문서 추가 - 수업에서는 하단에 있는 4줄 샘플로 진행
# 3) 문서→임베딩→DB (2교시 신규 추가)
os.makedirs("data", exist_ok=True)                                     # [추가]
doc_path = "data/정책.md"                                              # [추가]
if not os.path.exists(doc_path):                                       # [추가: 샘플 생성]
    with open(doc_path, "w", encoding="utf-8") as f:
        f.write(
            "- 디지털 상품은 원칙적으로 환불 불가 (결제 오류·중복 결제 예외)\n"
            "- 실물 상품은 수령 7일 이내 교환/환불 가능(미개봉)\n"
            "- VIP(GOLD) 고객은 1회 예외 환불 가능 (관리자 승인)\n"
            "- 배송 지연 3영업일 초과 시 사과 쿠폰 지급(내부 승인)\n"
        )

# 상단에 4줄 문장을 읽을 준비 완료
loader = TextLoader(doc_path, encoding = 'utf-8')
docs = loader.load()

# RAG관련 문서 정제
# 1) 문장을 청크(chunk)해서 나누고, 겹치게 구성
splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 100)
chunks = splitter.split_documents(docs)

# 2) 임베딩 - 문장을 숫자화
emb = OpenAIEmbeddings(model = 'text-embedding-3-small')
# 3) 임베딩된 문장을 Chroma DB에 저장! Vector DB에 저장
db  = Chroma.from_documents(chunks, emb)
# 향후 db에 있는 내용을 retriever로 가져오기만하면 되는 상황

# 자동 라우팅 규칙
# 사용자 질문에 해당 키워드가 있으면 RAG로 - 그렇지 않으면 일반 대화로
RAG_KEYWORDS = [
    "환불", "교환", "반품", "정책", "내규", "규정", "쿠폰", "배송", "VIP", "GOLD", "SILVER", "고객센터", "상담"
]

def route_to_rag(q):
    ql = q.lower()
    return any(kw.lower() in ql for kw in RAG_KEYWORDS)

# 위에 관련된 키워드가 등장했을때 RAG 챗봇이 등장!
def ask_rag(q):
    hits = db.similarity_search(q, k=3)
    context = "\n\n".join(d.page_content for d in hits)
    return rag_chain.invoke({'context' : context, "q": q})


# Streamlit 가장 중요한 요소 중 1개 chat_message - 스트림릿의 채팅 UI요소 뼈대
# session_state가 가지고 있는 메모리 -> 채팅UI 출력물을 뿌려줘야 하는데
# role 사람인지 챗봇인지 구별 -> content를 채팅 UI에 출력!
for m in st.session_state["messages"]:
    with st.chat_message(m['role']):
        st.markdown(m["content"])

# 알맹이 사용자 입력 -> LLM 응답 -> 렌더링(실시간 대화)
q = st.chat_input("질문을 입력해주세요") # input 설정
if q: #q가 질문이 입력이 되었다면
    with st.chat_message("user"):
        st.markdown(q) # q출력
    st.session_state['messages'].append({'role' : 'user', 'content' : q})

    # 조건부 라우팅 로직
    use_rag = route_to_rag(q) # 추가

    with st.chat_message('assistant'):
        if use_rag:
            ans = ask_rag(q) # 변경 - ask_rag를 활용한 답변
            st.markdown(ans)
            st.caption("문서기반 RAG로 답변 되었습니다.")
        else:
            ans = base_chain.invoke({"q": q})
            st.markdown(ans)
            st.caption("일반 LLM으로 답변 되었습니다.")
    st.session_state['messages'].append({'role' : 'assistant', 'content': ans})


Overwriting app.py


In [ ]:
# 4단계 - 근거 표시 출처, 하이라이트 기능 추가
%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # 2단계 추가
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader # 2단계 추가
from langchain_text_splitters import RecursiveCharacterTextSplitter # 2단계 추가
from langchain_community.vectorstores import Chroma # 2단계 추가

# 2단계에 필요한 라이브러리 추가

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RAG 챗봇 만들기 Step 4")
st.title("근거 표시 및 출처 기능 추가")

# 세션 초기화
# 임시 메모리 -> 새로고침 전까지 유지되는 메모리 - 대화 내용이 없으면 빈 리스트!
if "messages" not in st.session_state:
    st.session_state["messages"] = []

# LLM 구성
# 1. AI의 목적 및 프롬프트
# 2. 사용자의 대화를 전달받는 q {q}
# 3. 답변을 비워놔서 AI답변을 기다린다.
llm = ChatOpenAI(model = 'gpt-4o-mini')


# RAG 대화 전용 (이름 변경)
rag_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야.
아래 문서 내용을 참고해 질문에 답하세요.
문서   : {context}
사용자 : {q}
답변  :
""")
rag_chain = rag_prompt | llm | StrOutputParser()

# 일반 대화 전용 LCEL 구축 (일반 대화 - 추가)
base_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야
사용자 : {q}
답변  :
"""
)
base_chain = base_prompt | llm | StrOutputParser()


# RAG관련 문서 추가 - 수업에서는 하단에 있는 4줄 샘플로 진행
# 3) 문서→임베딩→DB (2교시 신규 추가)
os.makedirs("data", exist_ok=True)                                     # [추가]
doc_path = "data/정책.md"                                              # [추가]
if not os.path.exists(doc_path):                                       # [추가: 샘플 생성]
    with open(doc_path, "w", encoding="utf-8") as f:
        f.write(
            "- 디지털 상품은 원칙적으로 환불 불가 (결제 오류·중복 결제 예외)\n"
            "- 실물 상품은 수령 7일 이내 교환/환불 가능(미개봉)\n"
            "- VIP(GOLD) 고객은 1회 예외 환불 가능 (관리자 승인)\n"
            "- 배송 지연 3영업일 초과 시 사과 쿠폰 지급(내부 승인)\n"
        )

# 상단에 4줄 문장을 읽을 준비 완료
loader = TextLoader(doc_path, encoding = 'utf-8')
docs = loader.load()

# RAG관련 문서 정제
# 1) 문장을 청크(chunk)해서 나누고, 겹치게 구성
splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 100)
chunks = splitter.split_documents(docs)

# 2) 임베딩 - 문장을 숫자화
emb = OpenAIEmbeddings(model = 'text-embedding-3-small')
# 3) 임베딩된 문장을 Chroma DB에 저장! Vector DB에 저장
db  = Chroma.from_documents(chunks, emb)
# 향후 db에 있는 내용을 retriever로 가져오기만하면 되는 상황

# 자동 라우팅 규칙
# 사용자 질문에 해당 키워드가 있으면 RAG로 - 그렇지 않으면 일반 대화로
RAG_KEYWORDS = [
    "환불", "교환", "반품", "정책", "내규", "규정", "쿠폰", "배송", "VIP", "GOLD", "SILVER", "고객센터", "상담"
]

def route_to_rag(q):
    ql = q.lower()
    return any(kw.lower() in ql for kw in RAG_KEYWORDS)

# 근거를 표시하는 RAG 함수 구현~! (유사도 점수 + 근거 문장 표시)
def ask_rag_with_source(q):
    results = db.similarity_search_with_relevance_scores(q, k=3)
    if not results:
        return "관련 문서를 찾지 못했습니다."
    # 컨텍스트를 생성 -> 아규먼트 _추가 출처 result로 변경
    context = "\n\n".join(d.page_content for d, _ in results)

    # 답변을 생성을 기반으로 출처가 어디에서 나왔는지 확인하는 절차를 추가!
    # 1) 답변 생성
    answer = rag_chain.invoke({'context' : context, "q": q})
    # 2) 근거 목록 생성 (문단 일부 + 점수) - 핵심
    # source가 존재하면 -> 정책문서! / 유사도 점수를 기반으로 얼마나 연관되어있는지 / 실제 문서 근거
    sources = [
        f" {d.metadata.get('source',"정책문서")} 유사도 {score:.2f} \n {d.page_content[:120]}" for d, score in results
    ]

    return answer, sources


# Streamlit 가장 중요한 요소 중 1개 chat_message - 스트림릿의 채팅 UI요소 뼈대
# session_state가 가지고 있는 메모리 -> 채팅UI 출력물을 뿌려줘야 하는데
# role 사람인지 챗봇인지 구별 -> content를 채팅 UI에 출력!
for m in st.session_state["messages"]:
    with st.chat_message(m['role']):
        st.markdown(m["content"])

# 알맹이 사용자 입력 -> LLM 응답 -> 렌더링(실시간 대화)
q = st.chat_input("질문을 입력해주세요") # input 설정
if q: #q가 질문이 입력이 되었다면
    with st.chat_message("user"):
        st.markdown(q) # q출력
    st.session_state['messages'].append({'role' : 'user', 'content' : q})

    # 조건부 라우팅 로직
    use_rag = route_to_rag(q) # 추가

    with st.chat_message('assistant'):
        if use_rag:
            ans, sources = ask_rag_with_source(q) # 변경 - ask_rag_with_source를 실행해서 출처까지 답변
            st.markdown(ans)
            st.caption("문서기반 RAG로 답변 되었습니다.")

            if sources:
                st.markdown("참고근거")
                for s in sources:
                    st.markdown(s)

        else:
            ans = base_chain.invoke({"q": q})
            st.markdown(ans)
            st.caption("일반 LLM으로 답변 되었습니다.")
    st.session_state['messages'].append({'role' : 'assistant', 'content': ans})


Overwriting app.py


In [ ]:
# 5단계 - 고객 질의 RAG 챗봇 구현

%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # 2단계 추가
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader # 2단계 추가
from langchain_text_splitters import RecursiveCharacterTextSplitter # 2단계 추가
from langchain_community.vectorstores import Chroma # 2단계 추가
import pandas as pd  # 5단계 추가
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent # 5단계 추가

# 2단계에 필요한 라이브러리 추가

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RAG 챗봇 만들기 Step 5")
st.title("고객 DB기반 개인화 RAG 챗봇 구현")

# 세션 초기화
# 임시 메모리 -> 새로고침 전까지 유지되는 메모리 - 대화 내용이 없으면 빈 리스트!
if "messages" not in st.session_state:
    st.session_state["messages"] = []

# LLM 구성
# 1. AI의 목적 및 프롬프트
# 2. 사용자의 대화를 전달받는 q {q}
# 3. 답변을 비워놔서 AI답변을 기다린다.
llm = ChatOpenAI(model = 'gpt-4o-mini')

# 고객 CSV 파일 로드

# 경로 지정
os.makedirs("data", exist_ok = True)
csv_path = "data/customer_db.csv"
if not os.path.exists(csv_path):
    df_sample = pd.DataFrame([
        {"name" : "김민수", 'tier' : "GOLD", "purchase_count" : 23, "last_order" : "2025-10-20"},
        {"name" : "이지은", 'tier' : "SILVER", "purchase_count" : 5, "last_order" : "2025-10-23"},
        {"name" : "박철수", 'tier' : "VIP", "purchase_count" : 40, "last_order" : "2025-10-25"},
    ])
    df_sample.to_csv(csv_path, index=False, encoding ='utf-8')

df = pd.read_csv(csv_path, encoding = 'utf-8')
st.write("고객 DB 로드 완료", len(df), "명")

# 판다스 에이전트 생성
# df라는 데이터베이스 파일을 llm으로 읽어서 query기반으로 수행할 수 있는 에이전트 구현
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose = False,
    allow_dangerous_code = True,
    max_iterations = 10
)

# RAG 대화 전용 (이름 변경) - 고객정보 추가!
rag_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야.
아래 문서 내용을 참고해 질문에 답하세요.
고객정보 : {customer}
문서   : {context}
사용자 : {q}
답변  :
""")
rag_chain = rag_prompt | llm | StrOutputParser()

# 일반 대화 전용 LCEL 구축 (일반 대화 - 추가)
base_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야
사용자 : {q}
답변  :
"""
)
base_chain = base_prompt | llm | StrOutputParser()


# RAG관련 문서 추가 - 수업에서는 하단에 있는 4줄 샘플로 진행
# 3) 문서→임베딩→DB (2교시 신규 추가)
os.makedirs("data", exist_ok=True)                                     # [추가]
doc_path = "data/정책.md"                                              # [추가]
if not os.path.exists(doc_path):                                       # [추가: 샘플 생성]
    with open(doc_path, "w", encoding="utf-8") as f:
        f.write(
            "- 디지털 상품은 원칙적으로 환불 불가 (결제 오류·중복 결제 예외)\n"
            "- 실물 상품은 수령 7일 이내 교환/환불 가능(미개봉)\n"
            "- VIP(GOLD) 고객은 1회 예외 환불 가능 (관리자 승인)\n"
            "- 배송 지연 3영업일 초과 시 사과 쿠폰 지급(내부 승인)\n"
        )

# 상단에 4줄 문장을 읽을 준비 완료
loader = TextLoader(doc_path, encoding = 'utf-8')
docs = loader.load()

# RAG관련 문서 정제
# 1) 문장을 청크(chunk)해서 나누고, 겹치게 구성
splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 100)
chunks = splitter.split_documents(docs)

# 2) 임베딩 - 문장을 숫자화
emb = OpenAIEmbeddings(model = 'text-embedding-3-small')
# 3) 임베딩된 문장을 Chroma DB에 저장! Vector DB에 저장
db  = Chroma.from_documents(chunks, emb)
# 향후 db에 있는 내용을 retriever로 가져오기만하면 되는 상황

# 자동 라우팅 규칙
# 사용자 질문에 해당 키워드가 있으면 RAG로 - 그렇지 않으면 일반 대화로
RAG_KEYWORDS = [
    "환불", "교환", "반품", "정책", "내규", "규정", "쿠폰", "배송", "VIP", "GOLD", "SILVER", "고객센터", "상담", "등급"
]

def route_to_rag(q):
    ql = q.lower()
    return any(kw.lower() in ql for kw in RAG_KEYWORDS)

# 근거를 표시하는 RAG 함수 구현~! (유사도 점수 + 근거 문장 표시)
def personalized_rag(q):
    # 1. 고객 이름 추출 시도
    import re
    name = [n for n in df['name'] if n in q]
    if not name:
      return "고객 이름이 인식되지 않습니다. 예) 김민수 고객 환불 가능해?", []
    name = name[0]

    res = agent.invoke(f"{name}에 대한 고객 정보를 요약해줘")
    # invoke결과가 문자열 / 딕셔너리 형태로도 나올수 있어서 문자열이면 그냥 출력 -> 딕셔너리 일 경우에는 요약문자열을 출력해라
    customer_info = res if isinstance(res, str) else res.get("output", str(res))

    results = db.similarity_search_with_relevance_scores(q, k=3)
    if not results:
        return "관련 문서를 찾지 못했습니다.", []
    # 컨텍스트를 생성 -> 아규먼트 _추가 출처 result로 변경
    context = "\n\n".join(d.page_content for d, _ in results)

    # 답변을 생성을 기반으로 출처가 어디에서 나왔는지 확인하는 절차를 추가!
    # 1) 답변 생성
    answer = rag_chain.invoke({'customer': customer_info, 'context' : context, "q": q})
    # 2) 근거 목록 생성 (문단 일부 + 점수) - 핵심
    # source가 존재하면 -> 정책문서! / 유사도 점수를 기반으로 얼마나 연관되어있는지 / 실제 문서 근거
    sources = [
        f" {d.metadata.get('source',"정책문서")} 유사도 {score:.2f} \n {d.page_content[:120]}" for d, score in results
    ]

    return answer, sources


# Streamlit 가장 중요한 요소 중 1개 chat_message - 스트림릿의 채팅 UI요소 뼈대
# session_state가 가지고 있는 메모리 -> 채팅UI 출력물을 뿌려줘야 하는데
# role 사람인지 챗봇인지 구별 -> content를 채팅 UI에 출력!
for m in st.session_state["messages"]:
    with st.chat_message(m['role']):
        st.markdown(m["content"])

# 알맹이 사용자 입력 -> LLM 응답 -> 렌더링(실시간 대화)
q = st.chat_input("질문을 입력해주세요") # input 설정
if q: #q가 질문이 입력이 되었다면
    with st.chat_message("user"):
        st.markdown(q) # q출력
    st.session_state['messages'].append({'role' : 'user', 'content' : q})

    # 조건부 라우팅 로직
    use_rag = route_to_rag(q) # 추가

    with st.chat_message('assistant'):
        if use_rag:
            ans, sources = personalized_rag(q) # 변경 - personalized_rag를 실행해서 출처까지 답변
            st.markdown(ans)
            st.caption("DB, 문서기반 RAG로 답변 되었습니다.")

            if sources:
                st.markdown("참고근거")
                for s in sources:
                    st.markdown(s)

        else:
            ans = base_chain.invoke({"q": q})
            st.markdown(ans)
            st.caption("일반 LLM으로 답변 되었습니다.")
    st.session_state['messages'].append({'role' : 'assistant', 'content': ans})


Overwriting app.py


In [ ]:
# 6교시는 메모리 + UI + 속도 최적화

%%writefile app.py
import os, uuid
import streamlit as st
from langchain_openai import ChatOpenAI, OpenAIEmbeddings # 2단계 추가
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_community.document_loaders import TextLoader # 2단계 추가
from langchain_text_splitters import RecursiveCharacterTextSplitter # 2단계 추가
from langchain_community.vectorstores import Chroma # 2단계 추가
import pandas as pd  # 5단계 추가
from langchain_experimental.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent # 5단계 추가

# 2단계에 필요한 라이브러리 추가

# 기본 메인 화면 설정 페이지 타이틀 / 화면 타이틀
st.set_page_config(page_title = "RAG 챗봇 만들기 Step 6")
st.title("최종 RAG 챗봇 구현")

# 사이드바(UI) 옵션 추가
with st.sidebar:
    st.header("사이드바 옵션")
    k = st.slider("검색 개수 k", 1, 5, 2, help = "RAG 검색시 가져올 문서 수")
    show_sources = st.checkbox("근거(소스)표시, value = True")
    st.divider()
    c1, c2 = st.columns(2)
    if c1.button("새로운 대화 시작"):
        st.session_state.pop("messages", None) # 메세지도 초기화
        st.session_state.pop("history", None)  # 메모리도 초기화
        st.rerun() # 새로고침
    if c2.button("메모리 초기화"):
        st.session_state.pop("history", None) # 메모리만 초기화
        st.success("메모리가 초기화 되었습니다.")
    st.caption("이름/정책 키워드가 없으면 일반 대화로 자동 전환됩니다.")

# 세션 초기화
# 임시 메모리 -> 새로고침 전까지 유지되는 메모리 - 대화 내용이 없으면 빈 리스트!
if "messages" not in st.session_state:
    st.session_state["messages"] = []
if "history" not in st.session_state:
    st.session_state["history"] = []

# LLM 구성
# 1. AI의 목적 및 프롬프트
# 2. 사용자의 대화를 전달받는 q {q}
# 3. 답변을 비워놔서 AI답변을 기다린다.
llm = ChatOpenAI(model = 'gpt-4o-mini')

# 고객 CSV 파일 로드

# 경로 지정
os.makedirs("data", exist_ok = True)
csv_path = "data/customer_db.csv"
if not os.path.exists(csv_path):
    df_sample = pd.DataFrame([
        {"name" : "김민수", 'tier' : "GOLD", "purchase_count" : 23, "last_order" : "2025-10-20"},
        {"name" : "이지은", 'tier' : "SILVER", "purchase_count" : 5, "last_order" : "2025-10-23"},
        {"name" : "박철수", 'tier' : "VIP", "purchase_count" : 40, "last_order" : "2025-10-25"},
    ])
    df_sample.to_csv(csv_path, index=False, encoding ='utf-8')

df = pd.read_csv(csv_path, encoding = 'utf-8')
st.write("고객 DB 로드 완료", len(df), "명")

# 판다스 에이전트 생성
# df라는 데이터베이스 파일을 llm으로 읽어서 query기반으로 수행할 수 있는 에이전트 구현
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose = False,
    allow_dangerous_code = True,
    max_iterations = 10
)


# 최근 대화 n턴을 문자열로 만드는 기능
def get_history_text(n_turns = 6):
    hist = st.session_state['history'][-n_turns:] # 최근 정보부터 가져온다.
    lines = []
    for role, content in hist:
        prefix = "사용자" if role == "user" else "어시스턴트"
        lines.append(f"{prefix}: {content}")
    return "\n".join(lines) if lines else "대화이력 없음"


# RAG 대화 전용 (이름 변경) - 고객정보 추가!
rag_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야.
아래 문서 내용을 참고해 질문에 답하세요.
최근대화 : {history}
고객정보 : {customer}
문서   : {context}
사용자 : {q}
답변  :
""")
rag_chain = rag_prompt | llm | StrOutputParser()

# 일반 대화 전용 LCEL 구축 (일반 대화 - 추가)
base_prompt = ChatPromptTemplate.from_template("""
너는 친절한 한국어 비서야
최근대화 : {history}
사용자 : {q}
답변  :
"""
)
base_chain = base_prompt | llm | StrOutputParser()


# RAG관련 문서 추가 - 수업에서는 하단에 있는 4줄 샘플로 진행
# 3) 문서→임베딩→DB (2교시 신규 추가)
os.makedirs("data", exist_ok=True)                                     # [추가]
doc_path = "data/정책.md"                                              # [추가]
if not os.path.exists(doc_path):                                       # [추가: 샘플 생성]
    with open(doc_path, "w", encoding="utf-8") as f:
        f.write(
            "- 디지털 상품은 원칙적으로 환불 불가 (결제 오류·중복 결제 예외)\n"
            "- 실물 상품은 수령 7일 이내 교환/환불 가능(미개봉)\n"
            "- VIP(GOLD) 고객은 1회 예외 환불 가능 (관리자 승인)\n"
            "- 배송 지연 3영업일 초과 시 사과 쿠폰 지급(내부 승인)\n"
        )

# 상단에 4줄 문장을 읽을 준비 완료
loader = TextLoader(doc_path, encoding = 'utf-8')
docs = loader.load()

# RAG관련 문서 정제
# 1) 문장을 청크(chunk)해서 나누고, 겹치게 구성
splitter = RecursiveCharacterTextSplitter(chunk_size = 800, chunk_overlap = 100)
chunks = splitter.split_documents(docs)

# 2) 임베딩 - 문장을 숫자화
emb = OpenAIEmbeddings(model = 'text-embedding-3-small')
# 3) 임베딩된 문장을 Chroma DB에 저장! Vector DB에 저장
# -> 수정된 Chroma DB를 통해 데이터베이스를 저장해서 그 DB사용하는 방식으로 속도를 개선한다.
os.makedirs("db", exist_ok=True)
db  = Chroma.from_documents(chunks, emb, persist_directory="db")
db.persist()
# 향후 db에 있는 내용을 retriever로 가져오기만하면 되는 상황

# 자동 라우팅 규칙
# 사용자 질문에 해당 키워드가 있으면 RAG로 - 그렇지 않으면 일반 대화로
RAG_KEYWORDS = [
    "환불", "교환", "반품", "정책", "내규", "규정", "쿠폰", "배송", "VIP", "GOLD", "SILVER", "고객센터", "상담", "등급"
]

def route_to_rag(q):
    ql = q.lower()
    return any(kw.lower() in ql for kw in RAG_KEYWORDS)

# 근거를 표시하는 RAG 함수 구현~! (유사도 점수 + 근거 문장 표시)
def personalized_rag(q, k_override = None):
    # 1. 고객 이름 추출 시도
    import re
    name = [n for n in df['name'] if n in q]
    if not name:
      return "고객 이름이 인식되지 않습니다. 예) 김민수 고객 환불 가능해?", []
    name = name[0]

    res = agent.invoke(f"{name}에 대한 고객 정보를 요약해줘")
    # invoke결과가 문자열 / 딕셔너리 형태로도 나올수 있어서 문자열이면 그냥 출력 -> 딕셔너리 일 경우에는 요약문자열을 출력해라
    customer_info = res if isinstance(res, str) else res.get("output", str(res))

    # 슬라이드에 있는 k조절 기능으로 인해 따로 기능을 추가~!
    kk = k_override if k_override is not None else k
    results = db.similarity_search_with_relevance_scores(q, k=kk)

    if not results:
        return "관련 문서를 찾지 못했습니다.", []
    # 컨텍스트를 생성 -> 아규먼트 _추가 출처 result로 변경
    context = "\n\n".join(d.page_content for d, _ in results)

    history_text = get_history_text()

    # 답변을 생성을 기반으로 출처가 어디에서 나왔는지 확인하는 절차를 추가!
    # 1) 답변 생성
    answer = rag_chain.invoke({'history':history_text, 'customer': customer_info, 'context' : context, "q": q})
    # 2) 근거 목록 생성 (문단 일부 + 점수) - 핵심
    # source가 존재하면 -> 정책문서! / 유사도 점수를 기반으로 얼마나 연관되어있는지 / 실제 문서 근거
    sources = [
        f" {d.metadata.get('source',"정책문서")} 유사도 {score:.2f} \n {d.page_content[:120]}" for d, score in results
    ]

    return answer, sources


# Streamlit 가장 중요한 요소 중 1개 chat_message - 스트림릿의 채팅 UI요소 뼈대
# session_state가 가지고 있는 메모리 -> 채팅UI 출력물을 뿌려줘야 하는데
# role 사람인지 챗봇인지 구별 -> content를 채팅 UI에 출력!
for m in st.session_state["messages"]:
    with st.chat_message(m['role']):
        st.markdown(m["content"])

# 알맹이 사용자 입력 -> LLM 응답 -> 렌더링(실시간 대화)
q = st.chat_input("질문을 입력해주세요") # input 설정
if q: #q가 질문이 입력이 되었다면
    with st.chat_message("user"):
        st.markdown(q) # q출력
    st.session_state['messages'].append({'role' : 'user', 'content' : q})
    # 대화이력에 대한 히스토리 추가 (6교시 부분)
    st.session_state['history'].append(('user', 'q'))

    # 조건부 라우팅 로직
    use_rag = route_to_rag(q) # 추가

    with st.chat_message('assistant'):
        if use_rag:
            ans, sources = personalized_rag(q, k_override = k) # 변경 - personalized_rag를 실행해서 출처까지 답변
            st.markdown(ans)
            st.caption("DB, 문서기반 RAG로 답변 되었습니다.")

            if sources:
                st.markdown("참고근거")
                for s in sources:
                    st.markdown(s)

        else:
            history_text = get_history_text()
            ans = base_chain.invoke({"history" : history_text, "q": q})
            st.markdown(ans)
            st.caption("일반 LLM으로 답변 되었습니다.")
    st.session_state['messages'].append({'role' : 'assistant', 'content': ans})
    st.session_state['history'].append(('assistant', ans))


Overwriting app.py


In [ ]:
import os
from pyngrok import ngrok

# 서버를 자동 종료
try: ngrok.kill()
except: pass

os.system("pkill -f streamlit >/dev/null 2>&1 || true")
os.system("streamlit run app.py --server.address 0.0.0.0 --server.port 8501 &>/dev/null &")
print("🌐", ngrok.connect(addr="http://127.0.0.1:8501", bind_tls=True))


🌐 NgrokTunnel: "https://a3371c0d71bf.ngrok-free.app" -> "http://127.0.0.1:8501"
